# LangChain Expression Language (LCEL)

Recommended way to build language chain applications.

### Features

1. Parallel processing
2. Streaming
3. Batch Processing
4. Fallback in chains: with LCEL, any kind of chains can readilly incorporate fallbacks.
5. Integration with LangSmith as well

### Basic example: prompt + model + output parser
The most basic and common use case is chaining a prompt template and a model together. To see how this works, let’s create a chain that takes a topic and generates a joke:

In [1]:
# !pip install langchain==0.0.321

In [2]:
# !pip install langchain-core langchain-community langchain-openai

In [3]:
OPENAI_API_KEY=""

In [4]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

prompt = ChatPromptTemplate.from_template("tell me a short joke about {topic}")
model = ChatOpenAI(model="gpt-3.5-turbo",api_key=OPENAI_API_KEY)
output_parser = StrOutputParser()

chain = prompt | model | output_parser

chain.invoke({"topic": "ice cream"})

"Why did the ice cream break up with the cone? It couldn't handle the rocky road!"

The | symbol is similar to a unix pipe operator, which chains together the different components feeds the output from one component as input into the next component.

In above given situation, we passed input to the prompt, then we passed output of this prompt template to the model, then model output is passed to the output parser.

#### Prompt

Prompt is a BasePromptTemplate, which means it takes in a dictionary of template variables and produces a PromptValue. A PromptValue is a wrapper around a completed prompt that can be passed to either an LLM (which takes a string as input) or ChatModel (which takes a sequence of messages as input). It can work with either language model type because it defines logic both for producing BaseMessages and for producing a string.

In [5]:
prompt_value = prompt.invoke({"topic": "ice cream"})
prompt_value

ChatPromptValue(messages=[HumanMessage(content='tell me a short joke about ice cream')])

In [6]:
from langchain_core.prompt_values import ChatPromptValue, HumanMessage

In [7]:
ChatPromptValue(messages=[HumanMessage(content='tell me a short joke about ice cream')])

ChatPromptValue(messages=[HumanMessage(content='tell me a short joke about ice cream')])

In [8]:
prompt_value.to_messages()

[HumanMessage(content='tell me a short joke about ice cream')]

In [9]:
[HumanMessage(content='tell me a short joke about ice cream')]

[HumanMessage(content='tell me a short joke about ice cream')]

In [10]:
prompt_value.to_string()

'Human: tell me a short joke about ice cream'

#### Model
Output of this prompt is now in model. Here we used ChatModel, so output will be BaseMessage.

In [11]:
message = model.invoke(prompt_value)
message

AIMessage(content='Why did the ice cream go to therapy? Because it had too many sprinkles of self-doubt!', response_metadata={'token_usage': {'completion_tokens': 22, 'prompt_tokens': 15, 'total_tokens': 37}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': 'fp_3bc1b5746c', 'finish_reason': 'stop', 'logprobs': None})

In [12]:
from langchain_openai.llms import OpenAI

llm = OpenAI(model="gpt-3.5-turbo-instruct",api_key=OPENAI_API_KEY)
llm.invoke(prompt_value)

"\n\nWhy couldn't the ice cream go to the party? Because it was too cold to scoop."

#### Output parser

In [13]:
output_parser.invoke(message)

'Why did the ice cream go to therapy? Because it had too many sprinkles of self-doubt!'

#### Entire Pipeline

In [14]:
input = {"topic": "ice cream"}

prompt.invoke(input)

ChatPromptValue(messages=[HumanMessage(content='tell me a short joke about ice cream')])

In [15]:
(prompt | model).invoke(input)

AIMessage(content='Why did the ice cream truck break down?\nIt had too many "scoops"!', response_metadata={'token_usage': {'completion_tokens': 19, 'prompt_tokens': 15, 'total_tokens': 34}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': 'fp_3bc1b5746c', 'finish_reason': 'stop', 'logprobs': None})

### RAG Search Example

In [17]:
# pip install langchain docarray tiktoken

In [18]:
from langchain_community.vectorstores import DocArrayInMemorySearch
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableParallel, RunnablePassthrough
from langchain_openai.chat_models import ChatOpenAI
from langchain_openai.embeddings import OpenAIEmbeddings

vectorstore = DocArrayInMemorySearch.from_texts(
    ["harrison worked at kensho", "bears like to eat honey"],
    embedding=OpenAIEmbeddings(openai_api_key=OPENAI_API_KEY),
)
retriever = vectorstore.as_retriever()

template = """Answer the question based only on the following context:
{context}

Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)

model = ChatOpenAI(
    model="gpt-3.5-turbo",
    api_key=OPENAI_API_KEY,
    temperature=0.5
)
output_parser = StrOutputParser()

setup_and_retrieval = RunnableParallel(
    {"context": retriever, "question": RunnablePassthrough()}
)
chain = setup_and_retrieval | prompt | model | output_parser

chain.invoke("where did harrison work?")


2024-03-29 16:52:23.199025: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
/Users/c100-122/opt/anaconda3/lib/python3.9/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.4' currently installed).
  from pandas.core import (


'Harrison worked at Kensho.'

In [19]:
chain = setup_and_retrieval | prompt | model | output_parser

In [20]:
retriever.invoke("where did harrison work?")

[Document(page_content='harrison worked at kensho'),
 Document(page_content='bears like to eat honey')]

In [21]:
setup_and_retrieval = RunnableParallel(
    {"context": retriever, "question": RunnablePassthrough()}
)

In [22]:
question = "where did harrison work?"

In [23]:
setup_and_retrieval = RunnableParallel(
    {"context": retriever, "question": RunnablePassthrough()}
)
chain = setup_and_retrieval | prompt | model | output_parser

In [24]:
chain

{
  context: VectorStoreRetriever(tags=['DocArrayInMemorySearch'], vectorstore=<langchain_community.vectorstores.docarray.in_memory.DocArrayInMemorySearch object at 0x7f7c11e54520>),
  question: RunnablePassthrough()
}
| ChatPromptTemplate(input_variables=['context', 'question'], messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], template='Answer the question based only on the following context:\n{context}\n\nQuestion: {question}\n'))])
| ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x7f7c12c84af0>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x7f7c12c7c640>, temperature=0.5, openai_api_key=SecretStr('**********'), openai_proxy='')
| StrOutputParser()

In [25]:
result = chain.invoke(input=question)
print(result)

Harrison worked at Kensho.
